In [1]:
import cobra.test
import os
from os.path import join
from cobra.io import read_sbml_model
model = cobra.io.load_json_model(join("RehMBEL1391_sbml_L3V1.json"))
from cobra import Model, Reaction, Metabolite

# ADDING CO2 AS A CARBON SOURCE 

## AVIALABLE MEDIUM

In [2]:
model.medium

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

In [3]:
model.optimize()

,fluxes,reduced_costs
EX_gln__L_e,0.000000,-1.636524e-01
EX_galctn__D_e,0.000000,0.000000e+00
EX_c140_e,0.000000,-5.772063e-01
EX_15dap_e,0.000000,0.000000e+00
EX_3hbenzot_e,0.000000,-1.658639e-01
...,...,...
THZPSN,0.003577,-4.857226e-17
MTHF3ES_c,0.000000,-6.938894e-17
HYDS,0.000127,-2.081668e-17
EX_h2_e,0.000000,-1.548063e-02


## DEFINING THE NEW REACTIONS

Reaction: The extracellular CO2 is taken up to become cytoplasmic CO2.

In [4]:
co2exchange = Reaction('r_co2exchange')
co2exchange.name = 'CO2 Extracellular to Cytoplasm'
co2exchange.subsystem = 'CO2 Exchange'


In [5]:
co2_e = Metabolite(
    'co2_e',
    formula= 'co2_e',
    name= 'Carbon dioxide-extracellular',
    compartment= 'e')

In [6]:
co2_c = Metabolite(
    'co2_c',
    formula= 'co2',
    name= 'Carbon dioxide',
    compartment= 'c')


In [7]:
co2exchange.add_metabolites({
    co2_e: -1.0,
    model.metabolites.co2_c: 1.0,
})
co2exchange.reaction

'co2_e --> co2_c'

In [8]:
model.add_reactions([co2exchange])

In [9]:
model.reactions.get_by_id('r_co2exchange')

Reaction identifier,r_co2exchange
Name,CO2 Extracellular to Cytoplasm
Memory address,0x07f885506eb20
Stoichiometry,co2_e --> co2_c CO2 --> CO2
GPR,
Lower bound,0.0
Upper bound,1000.0


Once the CO2 is taken up by the cell the RUBISC enzyme uses it to carboxylate Rubilose 1,5-bisphosphate to give 3-Phosphoglycerate.
Now we create the reaction from CBB cycle.

In [10]:
rubisco = Reaction('r_rubisco')
rubisco.name = 'CO2 assimilation by CBB cycle'
rubisco.subsystem = 'CBB cycle'



In [11]:
co2_c = Metabolite(
    'co2_c',
    formula= 'co2',
    name= 'Carbon dioxide',
    compartment= 'c')


In [12]:
rubisco.add_metabolites({
    co2_c: -1.0,
    model.metabolites.rb15bp_c: -1.0,
    model.metabolites.get_by_id('3pg_c'): 1.0,
})
rubisco.reaction


'co2_c + rb15bp_c --> 3pg_c'

In [13]:
model.add_reactions([rubisco])

In [14]:
model.reactions.get_by_id('r_rubisco')

Reaction identifier,r_rubisco
Name,CO2 assimilation by CBB cycle
Memory address,0x07f885549a6d0
Stoichiometry,"co2_c + rb15bp_c --> 3pg_c CO2 + Ribulose-1,5-bisphosphate --> 3-Phospho-D-glycerate"
GPR,
Lower bound,0.0
Upper bound,1000.0


In [15]:
model.medium

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_fru_e': 3.0,
 'EX_nh4_e': 10.0,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

# CHANGING THE MEDIUM

The fructose in the medium is set to 0 and CO2 is is set to 10

In [16]:
medium = model.medium
medium['EX_co2_e'] = 10
medium['EX_fru_e'] = 0
model.medium = medium 
   

Checking the medium:

In [17]:
model.medium

{'EX_fe2_e': 10.0,
 'EX_mg2_e': 10.0,
 'EX_pi_e': 100.0,
 'EX_cobalt2_e': 10.0,
 'EX_cl_e': 10.0,
 'EX_k_e': 10.0,
 'EX_fe3_e': 10.0,
 'EX_so4_e': 10.0,
 'EX_nh4_e': 10.0,
 'EX_co2_e': 10,
 'EX_na_e': 10.0,
 'EX_o2_e': 18.5,
 'EX_mobd_e': 10.0,
 'EX_h2o_e': 1000.0,
 'EX_h_e': 100.0}

Calculating the values

In [18]:
solution= model.optimize()
print('The current theoretical maximum biomass productivity:', solution.fluxes['EX_BIOMASS_c'], 'h')

The current theoretical maximum biomass productivity: 0.0 h


In [19]:
model.objective = model.reactions.EX_pbhb_e
phb_production=model.optimize().objective_value
print('The current maximum theoretical productivity for making PHB:', phb_production , 'mmol/gDW*h')

The current maximum theoretical productivity for making PHB: 0.0 mmol/gDW*h


In [20]:
model.summary()

Metabolite,Reaction,Flux,C-Number,C-Flux
Metabolite,Reaction,Flux,C-Number,C-Flux


This doesn't seem to work. 